# Tugas Besar 1D - Pembelajaran Mesin


#### Anggota Kelompok
1. Ahmad Mutawalli - 13517026, K02
2. Harry Rahmadi Munly - 13517033, K03
3. Ardysatrio Fakhri Haroen - 13517062, K02
4. Dandi Agus Maulana - 13517077, K02

## Import Dependencies and Load Data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
import sys

from dtl.myC45 import Tree as MyC45
from mlp.MyMLP import Network as MyMLP

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load dataset
X_iris, y_iris = load_iris(return_X_y=True)
iris_data = pd.DataFrame(X_iris, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width']); iris_data['label'] = y_iris
iris_data = iris_data.sample(frac=1)
X_iris = iris_data.drop('label', axis=1)
y_iris = iris_data['label']
iris_data.head()

,sepal_length,sepal_width,petal_length,petal_width,label
66,5.6,3.0,4.5,1.5,1
8,4.4,2.9,1.4,0.2,0
12,4.8,3.0,1.4,0.1,0
108,6.7,2.5,5.8,1.8,2
11,4.8,3.4,1.6,0.2,0


## Define Functions and Procedures

In [3]:
#mencetak confusion matrix dengan layout yang mudah dilihat
def print_confusion_matrix(y_test, pred_result):
    cm = confusion_matrix(y_test, pred_result)
    cm = pd.DataFrame(cm, index=[i for i in range(3)], columns=[i for i in range(3)])
    sns.heatmap(cm, annot=True)
    plt.show()
    
#menghitung skor akurasi     
def accuracy_score(target, pred):
    sums = 0
    for i in range(len(pred)):
        if(pred[i] == target[i]):
           sums += 1
    return float(sums)/float(len(pred))

#Membagi data dengan cross-validation
def split_data_crossval(X, y, n_split=10):
    #Splitting data: KFold Cross Validation Indexing
    kfold = KFold(n_splits=n_split)
    split = kfold.split(X, y)

    train_indexes = []
    test_indexes = []
    for train, test in split:
        train_indexes.append(train)
        test_indexes.append(test)
    
    return train_indexes, test_indexes 

#melakukan iterasi k-fold cross validation
def cross_validate(X, y, model_name):
    
    #get train and test indexes
    train_indexes, test_indexes = split_data_crossval(X, y)

    scores = []
    #for each indexes in train_idx, train and test the model on it
    for train_idx, test_idx in zip(train_indexes, test_indexes):
        X_train = X[train_idx]; y_train = y[train_idx]
        X_test = X[test_idx]; y_test = y[test_idx]
        
        if(model_name == 'c45'):
            #train model
            train_data = pd.DataFrame(X_train, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
            train_data['label'] = pd.Series(y_train)
            
            test_data = pd.DataFrame(X_test, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
            test_data['label'] = pd.Series(y_test)
                        
            clf = MyC45(train_data, 'label')
            clf.make_tree()
            
            #get result
            pred_result = clf.predict(pd.DataFrame(test_data))
            #get accuracy
            scores.append(accuracy_score(pred_result, y_test))
            
        elif(model_name == 'mlp'):
            #train model
            net = MyMLP(4, 4)
            net.fit(X_train, y_train, epoch_limit=100)
            
            #get result
            pred_result = net.predict(X_test)
            #get accuracy
            scores.append(accuracy_score(pred_result, y_test))
            
    #return list of cross val scores
    return scores
               


## Train Test Split and Confusion Matrix

### Model C4.5

In [4]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.1, random_state=42)

#train model
train_data = pd.DataFrame(X_train, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
train_data['label'] = pd.Series(y_train)
            
test_data = pd.DataFrame(X_test, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
test_data['label'] = pd.Series(y_test)

clf = MyC45(train_data, 'label')
clf.make_tree()

#get result
pred_result = clf.predict(pd.DataFrame(test_data))

#print Accuracy
print("Prediction Accuracy: ", accuracy_score(y_test, pred_result) * 100 , "%", sep='')

#print confusion matrix
print("CONFUSION MATRIX (Row:True, Column:Prediction)")
print_confusion_matrix(y_test, pred_result)

KeyError: 0

### Model MLP

In [ ]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.1, random_state=42)

#train model
net = MyMLP(4, 4)
net.fit(X_train, y_train, epoch_limit=100)

#get result
pred_result = net.predict(X_test)

#print Accuracy
print("Prediction Accuracy: ", accuracy_score(y_test, pred_result) * 100 , "%", sep='')

#print confusion matrix
print("CONFUSION MATRIX (Row:True, Column:Prediction)")
print_confusion_matrix(y_test, pred_result)

## 10-fold Cross Validation

### Model C4.5

In [ ]:
c45_score = cross_validate(X_iris, y_iris, 'c45')
print("Average From 10-fold Cross Validation with C4.5:", np.mean(np.array(c45_score)))

### Model MLP

In [ ]:
mlp_score = cross_validate(X_iris, y_iris, 'mlp')
print("Average From 10-fold Cross Validation with MLP:", np.mean(np.array(mlp_score)))